**Train Test and Cross Validation**

In [1]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

print(f"🔍 SVM - Original dataset shape: {df.shape}")

df = df.dropna()

print(f"🧹 SVM - After dropping NaNs: {df.shape}")

X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"📊 SVM - Final dataset shape after scaling: {X_scaled.shape}")

svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

svm_kfold_scores = cross_val_score(svm, X_scaled, y, cv=kf, scoring='accuracy')

svm.fit(X_scaled, y)
y_pred = svm.predict(X_scaled)

print(f"✅ SVM - K-Fold Mean Accuracy: {svm_kfold_scores.mean():.4f}")
print("📌 SVM Classification Report:")
print(classification_report(y, y_pred))
print("🟦 SVM Confusion Matrix:")
print(confusion_matrix(y, y_pred))


🔍 SVM - Original dataset shape: (3268, 14)
🧹 SVM - After dropping NaNs: (3268, 14)
📊 SVM - Final dataset shape after scaling: (3268, 13)
✅ SVM - K-Fold Mean Accuracy: 0.7423
📌 SVM Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.68      0.75      1638
           1       0.73      0.86      0.79      1630

    accuracy                           0.77      3268
   macro avg       0.78      0.77      0.77      3268
weighted avg       0.78      0.77      0.77      3268

🟦 SVM Confusion Matrix:
[[1111  527]
 [ 226 1404]]


In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.0001],
}

grid_search = GridSearchCV(SVC(), param_grid, cv=kf, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_scaled, y)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[CV] END ...........................................C=0.0001; total time=   0.7s
[CV] END ...........................................C=0.0001; total time=   0.8s
[CV] END ...........................................C=0.0001; total time=   0.7s
[CV] END ...........................................C=0.0001; total time=   0.9s
[CV] END ...........................................C=0.0001; total time=   0.5s
Best Parameters: {'C': 0.0001}
Best Cross-Validation Accuracy: 0.4902


In [3]:
from sklearn.metrics import classification_report, confusion_matrix

valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"

df_valid = pd.read_csv(valid_path)
df_valid = df_valid.dropna()

X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

X_valid_scaled = scaler.transform(X_valid)

print(f"🧪 SVM - Validation dataset shape: {X_valid.shape}")

y_pred_valid = svm.predict(X_valid_scaled)

print(f"📌 SVM - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

print("🟦 SVM - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


🧪 SVM - Validation dataset shape: (364, 13)
📌 SVM - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.66      0.73       178
           1       0.72      0.84      0.78       186

    accuracy                           0.76       364
   macro avg       0.76      0.75      0.75       364
weighted avg       0.76      0.76      0.75       364

🟦 SVM - Validation Confusion Matrix:
[[118  60]
 [ 29 157]]
